In [2]:
import os
import cv2
from shutil import copyfile
import matplotlib.pyplot as plt
import csv
import random
import cv2
import numpy as np
from skimage import data
from skimage import exposure
from skimage.transform import match_histograms
from models import *
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
box_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity_new/unity_boxes"
unity_box_img_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box_img"
unity_box_mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box_mask"

# Unity

In [3]:
box_list = os.listdir(box_root)

In [4]:
for box in box_list:
    print(box)
    scene_path = os.path.join(box_root, box)
    scene_list = os.listdir(scene_path)
    
    unity_box_mask_scene_path = os.path.join(unity_box_mask_root, box)
    if not os.path.exists(unity_box_mask_scene_path):
        os.makedirs(unity_box_mask_scene_path)
        
    unity_box_img_scene_path = os.path.join(unity_box_img_root, box)
    if not os.path.exists(unity_box_img_scene_path):
        os.makedirs(unity_box_img_scene_path)
        
    for scene in scene_list:
        data_path = os.path.join(scene_path, scene)
        data_list = os.listdir(data_path)
        
        unity_box_mask_data_path = os.path.join(unity_box_mask_scene_path, scene)
        if not os.path.exists(unity_box_mask_data_path):
            os.makedirs(unity_box_mask_data_path)
        
        unity_box_img_data_path = os.path.join(unity_box_img_scene_path, scene)
        if not os.path.exists(unity_box_img_data_path):
            os.makedirs(unity_box_img_data_path)
        
        for data in data_list:
            if "_label" in data:
                mask_name = data
                img_name = data.split("_label")[0] + "_original.png"
                mask_path = os.path.join(data_path, mask_name)
                img_path = os.path.join(data_path, img_name)
                unity_box_mask_path = os.path.join(unity_box_mask_data_path, mask_name)
                unity_box_img_path = os.path.join(unity_box_img_data_path, img_name)
                copyfile(mask_path, unity_box_mask_path)
                copyfile(img_path, unity_box_img_path)

box_108
box_116
box_72


# GAN

In [3]:
cuda = True if torch.cuda.is_available() else False
G_AB = GeneratorResNet(res_blocks=9)
if cuda:
    G_AB = G_AB.cuda()
G_AB.load_state_dict(torch.load('/media/arg_ws3/5E703E3A703E18EB/research/cycle_box_sim2real/saved_models/box_sim2real_new/G_AB_579700_.pth'))
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

data_transform = transforms.Compose(
                [ transforms.Resize(int(480), Image.BICUBIC),
                #transforms.RandomCrop((opt.img_height, opt.img_width)),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
    )

In [4]:
def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.

    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.

    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_

    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.unsqueeze(0)
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.unsqueeze(0)

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new_full((3, height * ymaps + padding, width * xmaps + padding), pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid

In [5]:
def sample_images(image):
    """Saves a generated sample from the test set"""
    #imgs = next(iter(val_dataloader))
    #image = cv2.imread("/media/arg_ws3/5E703E3A703E18EB/data/mm_unity/unity_boxes/box_108/Scene65/11_original.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(image)
    pil_im = data_transform(pil_im)
    pil_im = pil_im.unsqueeze(0)

    image = cv2.resize(image, (256, 256)) 
    image = np.transpose(image, (2, 0, 1))
    image = torch.tensor(image)
    image = image.unsqueeze(0)

    #print(image.shape)
    #print(pil_im.shape)

    my_img = Variable(pil_im.type(Tensor))
    my_img_fake = G_AB(my_img)
    my_img_fake = my_img_fake.squeeze(0).detach().cpu()
    
    grid = make_grid(my_img_fake, nrow=5, normalize=True)
    pil_ = grid.mul_(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
    #print(pil_)
    pil_ = np.array(pil_)
    pil_ = pil_[...,::-1]
    pil_ = cv2.resize(pil_, (640, 480))
    return pil_
    #print(pil_.dtype)
    #cv2.imwrite("crop_mask1.jpg", pil_)

    #save_image(my_img_fake, 'boxxx.png', nrow=5, normalize=True)

In [ ]:
for box in box_list:
    mask_name = box.split('_original.png')[0] + '_seg.png'
    box_img = cv2.imread(os.path.join(box_root, box))
    matched = sample_images(box_img)
    image = cv2.imread(os.path.join(image_root, box))
    mask_img = cv2.imread(os.path.join(mask_root, mask_name), 0)
    mask_img[mask_img > 0] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    matched = cv2.bitwise_and(matched, mask, mask=mask_img)
    mask_img[mask_img < 255] = 1
    mask_img[mask_img == 255] = 0
    mask_img[mask_img == 1] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    image = cv2.bitwise_and(image, mask, mask=mask_img)
    result = cv2.add(matched, image)
    cv2.imwrite(os.path.join(root, box), result)

In [6]:
box_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity_new/unity_boxes"
box_gan_box_img_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/box_gan/box_img"
box_gan_box_mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/box_gan/box_mask"

In [7]:
box_list = os.listdir(box_root)

In [10]:
for box in box_list:
    print(box)
    scene_path = os.path.join(box_root, box)
    scene_list = os.listdir(scene_path)
    
    box_gan_box_mask_scene_path = os.path.join(box_gan_box_mask_root, box)
    if not os.path.exists(box_gan_box_mask_scene_path):
        os.makedirs(box_gan_box_mask_scene_path)
        
    box_gan_box_img_scene_path = os.path.join(box_gan_box_img_root, box)
    if not os.path.exists(box_gan_box_img_scene_path):
        os.makedirs(box_gan_box_img_scene_path)
        
    for scene in scene_list:
        data_path = os.path.join(scene_path, scene)
        data_list = os.listdir(data_path)
        
        box_gan_box_mask_data_path = os.path.join(box_gan_box_mask_scene_path, scene)
        if not os.path.exists(box_gan_box_mask_data_path):
            os.makedirs(box_gan_box_mask_data_path)
        
        box_gan_box_img_data_path = os.path.join(box_gan_box_img_scene_path, scene)
        if not os.path.exists(box_gan_box_img_data_path):
            os.makedirs(box_gan_box_img_data_path)
        
        for data in data_list:
            if "_label" in data:
                mask_name = data
                img_name = data.split("_label")[0] + "_original.png"
                mask_path = os.path.join(data_path, mask_name)
                img_path = os.path.join(data_path, img_name)
                box_gan_box_mask_path = os.path.join(box_gan_box_mask_data_path, mask_name)
                box_gan_box_img_path = os.path.join(box_gan_box_img_data_path, img_name)
                
                box_img = cv2.imread(img_path)
                matched = sample_images(box_img)
                
                cv2.imwrite(box_gan_box_img_path, matched)                
                copyfile(mask_path, box_gan_box_mask_path)

box_108
box_116
box_72


# Create CVS file

In [59]:
fcn_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/"
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/image/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/mask/"
image_list = os.listdir(image_root)
random.shuffle(image_list)
print(len(image_list))

68608


In [60]:
fcn_subt_root = '/media/arg_ws3/5E703E3A703E18EB/data/subt_all/'
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box_img/"
fcn_mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box_mask/"
data_list = []
training_box_list = []
test_box_list = []
count = 0
for obj in os.listdir(image_root):
    scene_path = os.path.join(image_root, obj)
    fcn_scene_path = os.path.join(fcn_mask_root, obj)
    scene_list = os.listdir(scene_path)
    for scene in scene_list:
        img_path = os.path.join(scene_path, scene)
        fcn_img_path = os.path.join(fcn_scene_path, scene)
        img_list = os.listdir(img_path)
        for img in img_list:
            file_path = os.path.join(img_path, img)
            fcn_file_path = os.path.join(fcn_img_path, img.split('_original')[0] + '_label.png')
            if os.path.exists(file_path) and os.path.exists(fcn_file_path):
                data_list.append(obj+'/'+scene+'/'+img)
            else:
                print(obj+'/'+scene+'/'+img)
random.shuffle(data_list)
print(len(data_list))

9472


In [61]:
all_data_list = image_list + data_list
random.shuffle(all_data_list)
print(len(all_data_list))

78080


In [62]:
all_training_list = []
all_testing_list = []
ratio = 0.9
count = 0
for data in all_data_list:
    if count < len(all_data_list)*ratio:
        all_training_list.append(data)
    else:
        all_testing_list.append(data)
    count = count + 1
print(len(all_training_list))
print(len(all_testing_list))

70272
7808


In [63]:
with open(fcn_root + 'train_with_box_all.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    counter = 0
    for image in all_training_list:
        counter = counter +1
        if image in image_list and image in data_list:
            print("Wrong!!! ", image)
        if image in image_list:
            mask_name = image.split("_original.png")[0] + "_seg.png"
            writer.writerow(["image/" + image, "mask/" + mask_name])
        elif image in data_list:
            mask_name = image.split("_original.png")[0] + "_label.png"
            writer.writerow(["box_img/" + image, "box_mask/" + mask_name])
        else:
            print("Wrong!!! ", image)
        if counter % 1000 == 0:
            print(counter)
with open(fcn_root + 'test_with_box_all.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    counter = 0
    for image in all_testing_list:
        counter = counter +1
        if image in image_list and image in data_list:
            print("Wrong!!! ", image)
        if image in image_list:
            mask_name = image.split("_original.png")[0] + "_seg.png"
            writer.writerow(["image/" + image, "mask/" + mask_name])
        elif image in data_list:
            mask_name = image.split("_original.png")[0] + "_label.png"
            writer.writerow(["box_img/" + image, "box_mask/" + mask_name])
        else:
            print("Wrong!!! ", image)
        if counter % 1000 == 0:
            print(counter)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
1000
2000
3000
4000
5000
6000
7000
